# Agentic LLM Systems - Practical Implementation

## Overview

Agentic LLM systems enable autonomous reasoning, planning, and action-taking capabilities. This notebook covers:

- **Planning Systems**: Hierarchical and dynamic planning algorithms
- **Reflection Mechanisms**: Self-assessment and meta-cognitive processes
- **ReAct Pattern**: Reasoning and Acting in synergistic loops
- **Multi-Agent Systems**: Coordination and collaboration between agents

Let's build practical implementations of these agentic capabilities.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import random
from datetime import datetime, timedelta
from collections import defaultdict, deque
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass
from enum import Enum
import asyncio

print("Libraries imported successfully!")

## 1. Hierarchical Planning System

Let's implement a hierarchical planning system that can break down complex goals into manageable sub-tasks:

In [ ]:
class TaskStatus(Enum):
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"
    FAILED = "failed"
    BLOCKED = "blocked"

@dataclass
class Task:
    id: str
    description: str
    priority: int
    estimated_duration: float
    dependencies: List[str]
    status: TaskStatus = TaskStatus.PENDING
    parent_task: Optional[str] = None
    subtasks: List[str] = None
    result: Optional[Dict] = None
    
    def __post_init__(self):
        if self.subtasks is None:
            self.subtasks = []

class HierarchicalPlanner:
    """Hierarchical task planning system for complex goal decomposition"""
    
    def __init__(self):
        self.tasks = {}
        self.execution_history = []
        self.planning_strategies = {
            'research': self.plan_research_task,
            'analysis': self.plan_analysis_task,
            'creation': self.plan_creation_task,
            'problem_solving': self.plan_problem_solving_task
        }
    
    def create_plan(self, goal_description, task_type='general', max_depth=3):
        """Create hierarchical plan for achieving a goal"""
        print(f"Creating plan for: {goal_description}")
        
        # Create root task
        root_task = Task(
            id="root",
            description=goal_description,
            priority=1,
            estimated_duration=0,
            dependencies=[]
        )
        
        self.tasks["root"] = root_task
        
        # Decompose into subtasks
        self.decompose_task("root", task_type, current_depth=0, max_depth=max_depth)
        
        # Create execution order
        execution_plan = self.create_execution_order()
        
        return {
            'root_task': root_task,
            'all_tasks': self.tasks,
            'execution_order': execution_plan,
            'total_tasks': len(self.tasks),
            'estimated_duration': self.calculate_total_duration()
        }
    
    def decompose_task(self, task_id, task_type, current_depth=0, max_depth=3):
        """Recursively decompose task into subtasks"""
        if current_depth >= max_depth:
            return
        
        task = self.tasks[task_id]
        
        # Use appropriate planning strategy
        if task_type in self.planning_strategies:
            subtasks = self.planning_strategies[task_type](task.description, current_depth)
        else:
            subtasks = self.plan_generic_task(task.description, current_depth)
        
        # Create subtask objects
        for i, subtask_desc in enumerate(subtasks):
            subtask_id = f"{task_id}_sub_{i+1}"
            
            subtask = Task(
                id=subtask_id,
                description=subtask_desc,
                priority=task.priority + 1,
                estimated_duration=random.uniform(0.5, 3.0),  # Simulate duration
                dependencies=[],
                parent_task=task_id
            )
            
            self.tasks[subtask_id] = subtask
            task.subtasks.append(subtask_id)
            
            # Recursively decompose if not at max depth
            if current_depth < max_depth - 1 and len(subtasks) <= 3:  # Limit branching
                self.decompose_task(subtask_id, task_type, current_depth + 1, max_depth)
    
    def plan_research_task(self, description, depth):
        """Planning strategy for research tasks"""
        if depth == 0:
            return [
                "Define research questions and scope",
                "Gather and review relevant sources",
                "Analyze and synthesize findings",
                "Document conclusions and insights"
            ]
        elif depth == 1:
            if "questions" in description.lower():
                return ["Identify key topics", "Formulate specific questions", "Prioritize research areas"]
            elif "sources" in description.lower():
                return ["Search academic databases", "Identify expert opinions", "Collect relevant data"]
            elif "analyze" in description.lower():
                return ["Compare different perspectives", "Identify patterns", "Draw connections"]
        return ["Execute research step", "Verify results", "Document findings"]
    
    def plan_analysis_task(self, description, depth):
        """Planning strategy for analysis tasks"""
        if depth == 0:
            return [
                "Collect and prepare data",
                "Apply analytical methods",
                "Interpret results",
                "Generate recommendations"
            ]
        elif depth == 1:
            if "data" in description.lower():
                return ["Clean and validate data", "Organize data structure", "Check data quality"]
            elif "methods" in description.lower():
                return ["Select appropriate techniques", "Run analysis", "Validate results"]
        return ["Perform analysis step", "Check accuracy", "Record outcomes"]
    
    def plan_creation_task(self, description, depth):
        """Planning strategy for creation tasks"""
        if depth == 0:
            return [
                "Plan and design approach",
                "Create initial version",
                "Review and refine",
                "Finalize and deliver"
            ]
        elif depth == 1:
            if "plan" in description.lower():
                return ["Define requirements", "Create outline", "Set milestones"]
            elif "create" in description.lower():
                return ["Build core components", "Add details", "Test functionality"]
        return ["Work on component", "Test and validate", "Integrate with whole"]
    
    def plan_problem_solving_task(self, description, depth):
        """Planning strategy for problem-solving tasks"""
        if depth == 0:
            return [
                "Understand and define the problem",
                "Generate potential solutions",
                "Evaluate and select best solution",
                "Implement and test solution"
            ]
        elif depth == 1:
            if "understand" in description.lower():
                return ["Gather problem details", "Identify constraints", "Define success criteria"]
            elif "generate" in description.lower():
                return ["Brainstorm ideas", "Research existing solutions", "Create novel approaches"]
        return ["Work on solution aspect", "Test approach", "Refine as needed"]
    
    def plan_generic_task(self, description, depth):
        """Generic planning strategy"""
        return [
            f"Prepare for: {description[:30]}...",
            f"Execute: {description[:30]}...",
            f"Verify: {description[:30]}..."
        ]
    
    def create_execution_order(self):
        """Create topologically sorted execution order"""
        # Simple depth-first traversal for demonstration
        execution_order = []
        visited = set()
        
        def dfs(task_id):
            if task_id in visited:
                return
            
            visited.add(task_id)
            task = self.tasks[task_id]
            
            # Visit subtasks first (depth-first)
            for subtask_id in task.subtasks:
                dfs(subtask_id)
            
            # Add current task after subtasks
            if not task.subtasks:  # Only add leaf tasks to execution order
                execution_order.append(task_id)
        
        dfs("root")
        return execution_order
    
    def calculate_total_duration(self):
        """Calculate total estimated duration"""
        leaf_tasks = [task for task in self.tasks.values() if not task.subtasks]
        return sum(task.estimated_duration for task in leaf_tasks)
    
    def execute_plan(self, execution_order, simulate=True):
        """Execute the planned tasks"""
        results = []
        
        print(f"Executing plan with {len(execution_order)} tasks...\n")
        
        for i, task_id in enumerate(execution_order, 1):
            task = self.tasks[task_id]
            
            print(f"Step {i}: {task.description}")
            
            if simulate:
                # Simulate task execution
                time.sleep(min(task.estimated_duration * 0.1, 0.5))  # Speed up for demo
                
                # Simulate success/failure
                success_probability = 0.9  # 90% success rate
                success = random.random() < success_probability
                
                if success:
                    task.status = TaskStatus.COMPLETED
                    result = f"Successfully completed: {task.description[:50]}..."
                    print(f"  ✓ {result}")
                else:
                    task.status = TaskStatus.FAILED
                    result = f"Failed to complete: {task.description[:50]}..."
                    print(f"  ✗ {result}")
                
                task.result = {'success': success, 'message': result}
                results.append(task.result)
            
            print()
        
        return results
    
    def get_plan_summary(self):
        """Get summary of the current plan"""
        total_tasks = len(self.tasks)
        leaf_tasks = [task for task in self.tasks.values() if not task.subtasks]
        completed_tasks = [task for task in self.tasks.values() if task.status == TaskStatus.COMPLETED]
        
        return {
            'total_tasks': total_tasks,
            'leaf_tasks': len(leaf_tasks),
            'completed_tasks': len(completed_tasks),
            'completion_rate': len(completed_tasks) / len(leaf_tasks) if leaf_tasks else 0,
            'estimated_duration': self.calculate_total_duration()
        }

# Initialize the planner
planner = HierarchicalPlanner()
print("Hierarchical planner initialized!")

### Testing Hierarchical Planning

Let's test our planner with different types of complex goals:

In [ ]:
# Test different planning scenarios
test_goals = [
    {
        'goal': 'Research the impact of artificial intelligence on healthcare',
        'type': 'research',
        'description': 'Complex research task'
    },
    {
        'goal': 'Analyze customer satisfaction data to improve service quality',
        'type': 'analysis',
        'description': 'Data analysis task'
    },
    {
        'goal': 'Create a comprehensive marketing strategy for a new product launch',
        'type': 'creation',
        'description': 'Creative planning task'
    }
]

planning_results = []

for i, test_case in enumerate(test_goals, 1):
    print(f"\n{'='*60}")
    print(f"TEST {i}: {test_case['description']}")
    print(f"{'='*60}")
    
    # Create fresh planner for each test
    test_planner = HierarchicalPlanner()
    
    # Create plan
    plan = test_planner.create_plan(
        test_case['goal'], 
        test_case['type'], 
        max_depth=3
    )
    
    print(f"\nPlan created successfully!")
    print(f"Total tasks: {plan['total_tasks']}")
    print(f"Execution steps: {len(plan['execution_order'])}")
    print(f"Estimated duration: {plan['estimated_duration']:.1f} hours")
    
    # Show task hierarchy
    print(f"\nTask Hierarchy:")
    test_planner.print_task_hierarchy("root", indent=0)
    
    # Execute plan
    print(f"\nExecuting plan...")
    execution_results = test_planner.execute_plan(plan['execution_order'], simulate=True)
    
    # Get summary
    summary = test_planner.get_plan_summary()
    
    print(f"\nExecution Summary:")
    print(f"  Completion rate: {summary['completion_rate']:.1%}")
    print(f"  Tasks completed: {summary['completed_tasks']}/{summary['leaf_tasks']}")
    
    planning_results.append({
        'goal_type': test_case['type'],
        'total_tasks': summary['total_tasks'],
        'completion_rate': summary['completion_rate'],
        'estimated_duration': summary['estimated_duration']
    })

# Add method to print hierarchy
def print_task_hierarchy(self, task_id, indent=0):
    """Print task hierarchy in tree format"""
    task = self.tasks[task_id]
    prefix = "  " * indent + ("├─ " if indent > 0 else "")
    status_symbol = {
        TaskStatus.COMPLETED: "✓",
        TaskStatus.FAILED: "✗",
        TaskStatus.IN_PROGRESS: "⟳",
        TaskStatus.PENDING: "○"
    }.get(task.status, "○")
    
    print(f"{prefix}{status_symbol} {task.description}")
    
    for subtask_id in task.subtasks:
        self.print_task_hierarchy(subtask_id, indent + 1)

# Add the method to the class
HierarchicalPlanner.print_task_hierarchy = print_task_hierarchy

# Summary visualization
df_results = pd.DataFrame(planning_results)

plt.figure(figsize=(12, 8))

# Task complexity by type
plt.subplot(2, 2, 1)
plt.bar(df_results['goal_type'], df_results['total_tasks'], color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Task Complexity by Goal Type')
plt.xlabel('Goal Type')
plt.ylabel('Total Tasks Generated')

# Completion rates
plt.subplot(2, 2, 2)
plt.bar(df_results['goal_type'], df_results['completion_rate'], color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Completion Rates by Goal Type')
plt.xlabel('Goal Type')
plt.ylabel('Completion Rate')
plt.ylim(0, 1)

# Duration estimates
plt.subplot(2, 2, 3)
plt.bar(df_results['goal_type'], df_results['estimated_duration'], color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Estimated Duration by Goal Type')
plt.xlabel('Goal Type')
plt.ylabel('Duration (hours)')

# Planning efficiency (tasks per hour)
plt.subplot(2, 2, 4)
efficiency = df_results['total_tasks'] / df_results['estimated_duration']
plt.bar(df_results['goal_type'], efficiency, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Planning Efficiency (Tasks/Hour)')
plt.xlabel('Goal Type')
plt.ylabel('Tasks per Hour')

plt.tight_layout()
plt.show()

print(f"\n=== PLANNING ANALYSIS ===")
print(f"Average tasks per goal: {df_results['total_tasks'].mean():.1f}")
print(f"Average completion rate: {df_results['completion_rate'].mean():.1%}")
print(f"Average duration: {df_results['estimated_duration'].mean():.1f} hours")

## 2. Reflection and Self-Assessment System

Let's implement a reflection system that allows agents to assess their own performance and improve:

In [ ]:
class ReflectionSystem:
    """Self-reflection and meta-cognitive system for agents"""
    
    def __init__(self):
        self.reflection_history = []
        self.performance_metrics = defaultdict(list)
        self.learning_insights = []
        self.improvement_strategies = []
        
        # Reflection frameworks
        self.reflection_frameworks = {
            'performance': self.reflect_on_performance,
            'process': self.reflect_on_process,
            'outcome': self.reflect_on_outcome,
            'learning': self.reflect_on_learning
        }
    
    def conduct_reflection(self, task_results, reflection_type='comprehensive'):
        """Conduct comprehensive reflection on task execution"""
        reflection_session = {
            'timestamp': datetime.now(),
            'task_results': task_results,
            'reflection_type': reflection_type,
            'insights': {},
            'action_items': [],
            'confidence_assessment': 0.0
        }
        
        if reflection_type == 'comprehensive':
            # Run all reflection frameworks
            for framework_name, framework_func in self.reflection_frameworks.items():
                insights = framework_func(task_results)
                reflection_session['insights'][framework_name] = insights
        else:
            # Run specific reflection framework
            if reflection_type in self.reflection_frameworks:
                insights = self.reflection_frameworks[reflection_type](task_results)
                reflection_session['insights'][reflection_type] = insights
        
        # Generate action items
        reflection_session['action_items'] = self.generate_action_items(reflection_session['insights'])
        
        # Assess confidence in reflection
        reflection_session['confidence_assessment'] = self.assess_reflection_confidence(reflection_session)
        
        # Store reflection
        self.reflection_history.append(reflection_session)
        
        return reflection_session
    
    def reflect_on_performance(self, task_results):
        """Reflect on performance metrics and outcomes"""
        successful_tasks = sum(1 for result in task_results if result.get('success', False))
        total_tasks = len(task_results)
        success_rate = successful_tasks / total_tasks if total_tasks > 0 else 0
        
        # Analyze performance patterns
        performance_insights = {
            'success_rate': success_rate,
            'total_tasks': total_tasks,
            'successful_tasks': successful_tasks,
            'performance_level': self.categorize_performance(success_rate),
            'strengths': [],
            'weaknesses': [],
            'trends': self.analyze_performance_trends()
        }
        
        # Identify strengths and weaknesses
        if success_rate > 0.8:
            performance_insights['strengths'].append("High task completion rate")
        elif success_rate < 0.6:
            performance_insights['weaknesses'].append("Low task completion rate")
        
        # Analyze failure patterns
        failed_tasks = [result for result in task_results if not result.get('success', False)]
        if failed_tasks:
            failure_analysis = self.analyze_failures(failed_tasks)
            performance_insights['failure_patterns'] = failure_analysis
        
        return performance_insights
    
    def reflect_on_process(self, task_results):
        """Reflect on the process and methodology used"""
        process_insights = {
            'efficiency_assessment': self.assess_process_efficiency(task_results),
            'bottlenecks': self.identify_bottlenecks(task_results),
            'optimization_opportunities': [],
            'process_quality': 'good'  # Simplified assessment
        }
        
        # Identify optimization opportunities
        if len(task_results) > 5:
            process_insights['optimization_opportunities'].append("Consider task batching for efficiency")
        
        # Assess process consistency
        consistency_score = self.assess_process_consistency(task_results)
        process_insights['consistency_score'] = consistency_score
        
        if consistency_score < 0.7:
            process_insights['optimization_opportunities'].append("Improve process standardization")
        
        return process_insights
    
    def reflect_on_outcome(self, task_results):
        """Reflect on the outcomes and their alignment with goals"""
        outcome_insights = {
            'goal_alignment': self.assess_goal_alignment(task_results),
            'quality_assessment': self.assess_outcome_quality(task_results),
            'impact_evaluation': self.evaluate_impact(task_results),
            'satisfaction_level': 'moderate'  # Simplified assessment
        }
        
        # Assess whether outcomes met expectations
        expected_outcomes = len(task_results)
        actual_successful_outcomes = sum(1 for result in task_results if result.get('success', False))
        
        outcome_insights['expectation_alignment'] = actual_successful_outcomes / expected_outcomes
        
        return outcome_insights
    
    def reflect_on_learning(self, task_results):
        """Reflect on learning and knowledge acquisition"""
        learning_insights = {
            'new_knowledge_acquired': self.identify_new_knowledge(task_results),
            'skills_developed': self.identify_skill_development(task_results),
            'knowledge_gaps': self.identify_knowledge_gaps(task_results),
            'learning_effectiveness': self.assess_learning_effectiveness(task_results)
        }
        
        # Track learning progress
        learning_insights['learning_trajectory'] = self.analyze_learning_trajectory()
        
        return learning_insights
    
    def categorize_performance(self, success_rate):
        """Categorize performance level"""
        if success_rate >= 0.9:
            return 'excellent'
        elif success_rate >= 0.8:
            return 'good'
        elif success_rate >= 0.6:
            return 'satisfactory'
        elif success_rate >= 0.4:
            return 'needs_improvement'
        else:
            return 'poor'
    
    def analyze_performance_trends(self):
        """Analyze performance trends over time"""
        if len(self.reflection_history) < 2:
            return {'trend': 'insufficient_data', 'direction': 'unknown'}
        
        recent_sessions = self.reflection_history[-3:]  # Last 3 sessions
        success_rates = []
        
        for session in recent_sessions:
            if 'performance' in session['insights']:
                success_rates.append(session['insights']['performance']['success_rate'])
        
        if len(success_rates) >= 2:
            if success_rates[-1] > success_rates[0]:
                return {'trend': 'improving', 'direction': 'upward'}
            elif success_rates[-1] < success_rates[0]:
                return {'trend': 'declining', 'direction': 'downward'}
            else:
                return {'trend': 'stable', 'direction': 'flat'}
        
        return {'trend': 'unknown', 'direction': 'unknown'}
    
    def analyze_failures(self, failed_tasks):
        """Analyze patterns in failed tasks"""
        failure_reasons = []
        
        for task in failed_tasks:
            # Simulate failure reason analysis
            possible_reasons = [
                'insufficient_information',
                'complexity_too_high',
                'resource_constraints',
                'external_dependencies',
                'unclear_requirements'
            ]
            failure_reasons.append(random.choice(possible_reasons))
        
        # Count failure reason frequency
        reason_counts = Counter(failure_reasons)
        
        return {
            'total_failures': len(failed_tasks),
            'common_reasons': dict(reason_counts.most_common(3)),
            'failure_rate': len(failed_tasks) / (len(failed_tasks) + 10)  # Assuming some successful tasks
        }
    
    def generate_action_items(self, insights):
        """Generate actionable improvement items based on insights"""
        action_items = []
        
        # Performance-based actions
        if 'performance' in insights:
            perf = insights['performance']
            if perf['success_rate'] < 0.7:
                action_items.append({
                    'category': 'performance',
                    'action': 'Improve task completion strategies',
                    'priority': 'high',
                    'timeline': 'immediate'
                })
        
        # Process-based actions
        if 'process' in insights:
            process = insights['process']
            if process.get('consistency_score', 1.0) < 0.7:
                action_items.append({
                    'category': 'process',
                    'action': 'Standardize task execution procedures',
                    'priority': 'medium',
                    'timeline': 'short_term'
                })
        
        # Learning-based actions
        if 'learning' in insights:
            learning = insights['learning']
            if learning.get('knowledge_gaps'):
                action_items.append({
                    'category': 'learning',
                    'action': 'Address identified knowledge gaps',
                    'priority': 'medium',
                    'timeline': 'medium_term'
                })
        
        return action_items
    
    def assess_reflection_confidence(self, reflection_session):
        """Assess confidence in the reflection analysis"""
        # Simple confidence assessment based on data availability
        data_points = len(reflection_session['task_results'])
        insight_depth = len(reflection_session['insights'])
        
        confidence = min(1.0, (data_points * 0.1) + (insight_depth * 0.2))
        return confidence
    
    # Simplified implementations for demonstration
    def assess_process_efficiency(self, task_results):
        return random.uniform(0.6, 0.9)
    
    def identify_bottlenecks(self, task_results):
        return ['task_complexity', 'resource_availability']
    
    def assess_process_consistency(self, task_results):
        return random.uniform(0.5, 0.9)
    
    def assess_goal_alignment(self, task_results):
        return random.uniform(0.7, 0.95)
    
    def assess_outcome_quality(self, task_results):
        return random.uniform(0.6, 0.9)
    
    def evaluate_impact(self, task_results):
        return {'impact_level': 'moderate', 'stakeholder_satisfaction': 0.8}
    
    def identify_new_knowledge(self, task_results):
        return ['domain_specific_insights', 'process_improvements']
    
    def identify_skill_development(self, task_results):
        return ['problem_solving', 'analytical_thinking']
    
    def identify_knowledge_gaps(self, task_results):
        return ['advanced_techniques', 'domain_expertise']
    
    def assess_learning_effectiveness(self, task_results):
        return random.uniform(0.6, 0.9)
    
    def analyze_learning_trajectory(self):
        return {'direction': 'positive', 'rate': 'steady'}

# Initialize reflection system
reflection_system = ReflectionSystem()
print("Reflection system initialized!")

### Testing Reflection System

Let's test the reflection system with simulated task results:

In [ ]:
# Simulate different task execution scenarios
test_scenarios = [
    {
        'name': 'High Performance Scenario',
        'task_results': [
            {'success': True, 'message': 'Task 1 completed successfully'},
            {'success': True, 'message': 'Task 2 completed successfully'},
            {'success': True, 'message': 'Task 3 completed successfully'},
            {'success': True, 'message': 'Task 4 completed successfully'},
            {'success': False, 'message': 'Task 5 failed due to complexity'}
        ]
    },
    {
        'name': 'Mixed Performance Scenario',
        'task_results': [
            {'success': True, 'message': 'Task 1 completed successfully'},
            {'success': False, 'message': 'Task 2 failed - insufficient data'},
            {'success': True, 'message': 'Task 3 completed successfully'},
            {'success': False, 'message': 'Task 4 failed - resource constraints'},
            {'success': True, 'message': 'Task 5 completed successfully'}
        ]
    },
    {
        'name': 'Low Performance Scenario',
        'task_results': [
            {'success': False, 'message': 'Task 1 failed - unclear requirements'},
            {'success': False, 'message': 'Task 2 failed - complexity too high'},
            {'success': True, 'message': 'Task 3 completed successfully'},
            {'success': False, 'message': 'Task 4 failed - external dependencies'},
            {'success': False, 'message': 'Task 5 failed - insufficient information'}
        ]
    }
]

reflection_results = []

for scenario in test_scenarios:
    print(f"\n{'='*60}")
    print(f"REFLECTION SESSION: {scenario['name']}")
    print(f"{'='*60}")
    
    # Conduct reflection
    reflection = reflection_system.conduct_reflection(
        scenario['task_results'], 
        reflection_type='comprehensive'
    )
    
    print(f"\nReflection completed with confidence: {reflection['confidence_assessment']:.2f}")
    
    # Display performance insights
    if 'performance' in reflection['insights']:
        perf = reflection['insights']['performance']
        print(f"\nPerformance Analysis:")
        print(f"  Success Rate: {perf['success_rate']:.1%}")
        print(f"  Performance Level: {perf['performance_level']}")
        print(f"  Successful Tasks: {perf['successful_tasks']}/{perf['total_tasks']}")
        
        if perf['strengths']:
            print(f"  Strengths: {', '.join(perf['strengths'])}")
        if perf['weaknesses']:
            print(f"  Weaknesses: {', '.join(perf['weaknesses'])}")
        
        print(f"  Trend: {perf['trends']['trend']} ({perf['trends']['direction']})")
    
    # Display process insights
    if 'process' in reflection['insights']:
        process = reflection['insights']['process']
        print(f"\nProcess Analysis:")
        print(f"  Efficiency: {process['efficiency_assessment']:.2f}")
        print(f"  Consistency: {process['consistency_score']:.2f}")
        print(f"  Quality: {process['process_quality']}")
        
        if process['optimization_opportunities']:
            print(f"  Optimization Opportunities:")
            for opp in process['optimization_opportunities']:
                print(f"    - {opp}")
    
    # Display learning insights
    if 'learning' in reflection['insights']:
        learning = reflection['insights']['learning']
        print(f"\nLearning Analysis:")
        print(f"  Learning Effectiveness: {learning['learning_effectiveness']:.2f}")
        print(f"  New Knowledge: {', '.join(learning['new_knowledge_acquired'])}")
        print(f"  Skills Developed: {', '.join(learning['skills_developed'])}")
        print(f"  Knowledge Gaps: {', '.join(learning['knowledge_gaps'])}")
    
    # Display action items
    print(f"\nAction Items ({len(reflection['action_items'])}):") 
    for i, action in enumerate(reflection['action_items'], 1):
        print(f"  {i}. [{action['priority'].upper()}] {action['action']}")
        print(f"     Category: {action['category']}, Timeline: {action['timeline']}")
    
    # Store results for analysis
    reflection_results.append({
        'scenario': scenario['name'],
        'success_rate': reflection['insights']['performance']['success_rate'],
        'confidence': reflection['confidence_assessment'],
        'action_items': len(reflection['action_items']),
        'performance_level': reflection['insights']['performance']['performance_level']
    })

# Visualize reflection analysis
df_reflections = pd.DataFrame(reflection_results)

plt.figure(figsize=(15, 10))

# Success rates by scenario
plt.subplot(2, 3, 1)
colors = ['green', 'orange', 'red']
bars = plt.bar(range(len(df_reflections)), df_reflections['success_rate'], color=colors)
plt.title('Success Rates by Scenario')
plt.xlabel('Scenario')
plt.ylabel('Success Rate')
plt.xticks(range(len(df_reflections)), ['High', 'Mixed', 'Low'], rotation=45)
plt.ylim(0, 1)

# Add percentage labels
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.1%}', ha='center', va='bottom')

# Reflection confidence
plt.subplot(2, 3, 2)
plt.bar(range(len(df_reflections)), df_reflections['confidence'], color=colors)
plt.title('Reflection Confidence')
plt.xlabel('Scenario')
plt.ylabel('Confidence Score')
plt.xticks(range(len(df_reflections)), ['High', 'Mixed', 'Low'], rotation=45)
plt.ylim(0, 1)

# Action items generated
plt.subplot(2, 3, 3)
plt.bar(range(len(df_reflections)), df_reflections['action_items'], color=colors)
plt.title('Action Items Generated')
plt.xlabel('Scenario')
plt.ylabel('Number of Action Items')
plt.xticks(range(len(df_reflections)), ['High', 'Mixed', 'Low'], rotation=45)

# Performance level distribution
plt.subplot(2, 3, 4)
performance_levels = df_reflections['performance_level'].value_counts()
plt.pie(performance_levels.values, labels=performance_levels.index, autopct='%1.0f%%')
plt.title('Performance Level Distribution')

# Correlation between success rate and confidence
plt.subplot(2, 3, 5)
plt.scatter(df_reflections['success_rate'], df_reflections['confidence'], 
           c=colors, s=100, alpha=0.7)
plt.xlabel('Success Rate')
plt.ylabel('Reflection Confidence')
plt.title('Success Rate vs Reflection Confidence')

# Add scenario labels
for i, row in df_reflections.iterrows():
    plt.annotate(['High', 'Mixed', 'Low'][i], 
                (row['success_rate'], row['confidence']),
                xytext=(5, 5), textcoords='offset points')

# Reflection effectiveness over time
plt.subplot(2, 3, 6)
sessions = range(1, len(reflection_system.reflection_history) + 1)
confidences = [session['confidence_assessment'] for session in reflection_system.reflection_history]
plt.plot(sessions, confidences, marker='o', linewidth=2, markersize=6)
plt.title('Reflection Confidence Over Time')
plt.xlabel('Reflection Session')
plt.ylabel('Confidence Score')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n=== REFLECTION SYSTEM ANALYSIS ===")
print(f"Total reflection sessions: {len(reflection_system.reflection_history)}")
print(f"Average reflection confidence: {np.mean([s['confidence_assessment'] for s in reflection_system.reflection_history]):.3f}")
print(f"Average action items per session: {df_reflections['action_items'].mean():.1f}")
print(f"Performance improvement correlation: {np.corrcoef(df_reflections['success_rate'], df_reflections['confidence'])[0,1]:.3f}")